In [9]:
#ETL USING PYSPARK

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, min, avg, count, concat_ws, col
spark = SparkSession.builder.appName("ETL using Pyspark").getOrCreate()

In [3]:
#EXTRACT
#Data Extraction
data_pricing_1_df = spark.read.option("header",True).option("inferSchema",True).csv("Data pricing 1.csv")
data_pricing_2_df = spark.read.option("header",True).option("inferSchema",True).csv("Data pricing 2.csv")

In [4]:
#Data Integration
data_pricing_df = data_pricing_1_df.union(data_pricing_2_df)

In [5]:
data_pricing_df = data_pricing_df.withColumn("ID",concat_ws("_",col("Continental region")[0:5],col("Currency")))

In [8]:
data_pricing_df = data_pricing_df.withColumn("CHEAP_WITH_AVG",col("Cheapest 1GB (Local currency)") + col("Average price of 1GB (local currency)"))

In [16]:
data_pricing_df = data_pricing_df.withColumnRenamed("Plans measured","Plans")
data_pricing_df = data_pricing_df.select(col("ID"), col("Plans"), col("Continental region"))

AnalysisException: cannot resolve '`Continental region`' given input columns: [ID, Plans];
'Project [ID#163, Plans#220, 'Continental region]
+- Project [ID#163, Plans#220]
   +- Project [Continental region#92, Plans measured#93 AS Plans#220, Average price of 1GB (local currency)#94, Currency#95, Conversion rate (USD) (Frozen 07/09/2023)#96, Average price of 1GB (USD)#97, Cheapest 1GB (Local currency)#98, Cheapest 1GB for 30 days (USD)#99, Most expensive 1GB (Local currency)#100, Most expensive 1GB (USD)#101, DATE#102, ID#163, CHEAP_WITH_AVG#206]
      +- Project [Continental region#92, Plans measured#93, Average price of 1GB (local currency)#94, Currency#95, Conversion rate (USD) (Frozen 07/09/2023)#96, Average price of 1GB (USD)#97, Cheapest 1GB (Local currency)#98, Cheapest 1GB for 30 days (USD)#99, Most expensive 1GB (Local currency)#100, Most expensive 1GB (USD)#101, DATE#102, ID#163, (Cheapest 1GB (Local currency)#98 + Average price of 1GB (local currency)#94) AS CHEAP_WITH_AVG#20

In [15]:
data_pricing_df.write.mode("overwrite").partitionBy("Continental region").parquet("data priciing.parquet")

AnalysisException: Partition column `Continental region` not found in schema struct<ID:string,Plans:int>